## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Clinical Deidentification Portuguese

Deidentification is essential for safeguarding patient privacy in clinical data, including texts, PDFs, images, and DICOM files containing Protected Health Information (PHI). PHI encompasses various health-related data, including common identifiers such as name, address, birth date, and Social Security Number.

- **Model**: `pt.deid.clinical`
- **Model Description**: This pipeline is trained with w2v_cc_300d portuguese embeddings and can be used to deidentify PHI information from medical texts in portuguese. The PHI information will be masked and obfuscated in the resulting text. The pipeline can mask, fake or obfuscate the following entities: AGE, DATE, PROFESSION, EMAIL, ID, COUNTRY, STREET, DOCTOR, HOSPITAL, PATIENT, URL, IP, ORGANIZATION, PHONE, ZIP, ACCOUNT, SSN, PLATE, SEX and IPADDR

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "pt-deid-clinical"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"



### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

--------!

Once endpoint has been created, you would be able to perform real-time inference.

In [6]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'


    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)
    
    input_data = json.dumps(data, indent=4, ensure_ascii=False) if content_type == 'application/json' else data


    if content_type == 'application/json':
        with open(input_file_name, "w", encoding='utf-8') as f:
            f.write(input_data)
    else:
        with open(input_file_name, 'w', encoding='utf-8') as file:
            for line in input_data.splitlines():
                json_object = json.loads(line)
                file.write(json.dumps(json_object, ensure_ascii=False) + '\n')

    with open(input_file_name, 'rb') as data:
        s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=data)

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4, ensure_ascii=False)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    with open(output_file_name, 'rb') as data:
        s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_output_dir}/{os.path.basename(output_file_name)}", Body=data)

    return response_data

### Initial Setup

In [7]:
docs = [
'''Dados do paciente.
Nome: Mauro.
Apelido: Gonçalves.
NISS: 26 63514095.
Endereço: Calle Miguel Benitez 90.
CÓDIGO POSTAL: 28016.
Dados de cuidados.
Data de nascimento: 03/03/1946.
País: Portugal.
Idade: 70 anos Sexo: M.
Data de admissão: 12/12/2016.
Doutor: Ignacio Navarro Cuéllar NºCol: 28 28 70973.
Relatório clínico do paciente: Paciente de 70 anos, mineiro reformado, sem alergias medicamentosas conhecidas, que apresenta como história pessoal: acidente de trabalho antigo com fracturas vertebrais e das costelas; operado por doença de Dupuytren na mão direita e iliofemoral esquerda; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; alcoolismo activo, fumador de 20 cigarros / dia.
Foi encaminhado dos cuidados primários porque apresentou uma vez hematúria macroscópica pós-morte e depois microhaematúria persistente, com micturição normal.
O exame físico mostrou um bom estado geral, com abdómen e genitália normais; o exame rectal foi compatível com adenoma de próstata de grau I/IV.
A urinálise mostrou 4 glóbulos vermelhos/campo e 0-5 leucócitos/campo; o resto do sedimento estava normal.
Hemograma normal; a bioquímica mostrou glicemia de 169 mg/dl e triglicéridos de 456 mg/dl; função hepática e renal normal. PSA de 1,16 ng/ml.
A citologia da urina era repetidamente desconfiada por malignidade.
A radiografia simples abdominal mostra alterações degenerativas na coluna lombar e calcificações vasculares tanto no hipocôndrio como na pélvis.
A ecografia urológica revelou cistos corticais simples no rim direito, uma bexiga inalterada com boa capacidade e uma próstata com 30g de peso.
O IVUS mostrou normofuncionalismo renal bilateral, calcificações na silhueta renal direita e ureteres artrosados com imagens de adição no terço superior de ambos os ureteres, relacionadas com pseudodiverticulose ureteral. O cistograma mostra uma bexiga com boa capacidade, mas com paredes trabeculadas em relação à bexiga de stress. A tomografia computorizada abdominal é normal.
A cistoscopia revelou a existência de pequenos tumores na bexiga, e a ressecção transuretral foi realizada com o resultado anatomopatológico do carcinoma urotelial superficial da bexiga.
Referido por: Miguel Santos - Avenida dos Aliados, 22 Portugal E-mail: nnavcu@hotmail.com.''',

'''RELAÇÃO HOSPITALAR
NOME: Pedro Gonçalves
NHC: MVANSK92F09W408A
ENDEREÇO: Rua Burcardo 7
CÓDIGO POSTA... 
RELAÇÃO HOSPITALAR
NOME: Pedro Gonçalves
NHC: MVANSK92F09W408A
ENDEREÇO: Rua Burcardo 7
CÓDIGO POSTAL: 80139
DATA DE NASCIMENTO: 03/03/1946
IDADE: 70 anos
SEXO: Homens
E-MAIL: pgon21@tim.pt
DATA DE ADMISSÃO: 12/12/2016
DOUTORA: Eva Andrade
RELATO CLÍNICO: 70 anos, aposentado, sem alergia a medicamentos conhecida, com a seguinte história: ex-acidente de trabalho com fratura de vértebras e costelas; operado de doença de Dupuytren na mão direita e ponte ílio-femoral esquerda; diabetes tipo II, hipercolesterolemia e hiperuricemia; alcoolismo ativo, fuma 20 cigarros/dia.
Ele foi-nos encaminhado por apresentar hematúria macroscópica pós-evacuação em uma ocasião e microhematúria persistente posteriormente, com evacuação normal.
O exame físico mostrou bom estado geral, com abdome e genitais normais; o toque retal foi compatível com adenoma de próstata grau I/IV.
A urinálise mostrou 4 hemácias/campo e 0-5 leucócitos/campo; o resto do sedimento era normal.
O hemograma é normal; a bioquímica mostrou uma glicemia de 169 mg/dl e triglicerídeos 456 mg/dl; função hepática e renal são normais. PSA de 1,16 ng/ml.

DIRIGIDA A: Dr. Eva Andrade - Centro Hospitalar do Medio Ave - Avenida Dos Aliados, 56
E-MAIL: evandrade@poste.pt'''

]


sample_text = """Nome do paciente: Antonio Gonçalves
NHC: 2569870.
Endereço: Rua Das Flores 23.
Código Postal: 21754-... 
Nome do paciente: Antonio Gonçalves
NHC: 2569870.
Endereço: Rua Das Flores 23.
Código Postal: 21754-987.
Dados de cuidados.
Data de nascimento: 10/10/1963.
Idade: 23 anos 
Data de admissão: 17/06/2016.
Doutora: Maria Santos"""

### Important Parameters

- **masking_policy**: `str`

    Users can select a masking policy to determine how sensitive entities are handled:

    Example: "**Dados do paciente. Nome: Mauro. Apelido: Gonçalves. NISS: 26 63514095. Endereço: Calle Miguel Benitez 90.**"

    - **masked**: Default policy that masks entities with their type.

      -> 'Dados do `<SEX>`. Nome: `<PATIENT>`. Apelido: `<PATIENT>`. NISS: `<ID>`. Endereço: `<STREET>`.'

    - **obfuscated**: Replaces sensitive entities with random values of the same type.

      -> 'Dados do `Homen`. Nome: `Josefina Batista`. Apelido: `Tiago Santos`. NISS: `16 63514095`. Endereço: `Avenida da Liberdade, 17`.'

    - **masked_fixed_length_chars**: Masks entities with a fixed length of asterisks (\*).

      -> 'Dados do `****`. Nome: `****`. Apelido: `****`. NISS: `****`. Endereço: `****`.'

    - **masked_with_chars**: Masks entities with asterisks (\*).

      -> 'Dados do [`******`]. Nome: [`***`]. Apelido: [`*******`]. NISS: [`**********`]. Endereço: [`*********************`].'

- **sep**: `str`

    Separator used to join subparts within each prediction.

    By default, the separator is set to a single space (" "), but users can specify any other separator as needed. Necessary because the model outputs predictions as separate subparts, and the chosen separator is used to join them into coherent text.

    The separator must be one of the following characters: space (' '), newline ('\n'), comma (','), tab ('\t'), or colon (':').
    
You can specify these parameters in the input as follows:

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ],
    "masking_policy": "masked",
    "sep": " ",
}
```

### JSON


1. **Input format**: Single Text Document

Provide a single text document as a string.

  
  
```json
{
    "text": "Single text document"
}
```

2. **Input format**: Array of Text Documents

Use an array containing multiple text documents. Each element represents a separate text document.

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

#### Example 1: masked (default-policy)

In [8]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )

pd.DataFrame(data)

,predictions
0,Nome do <SEX>: <PATIENT> NHC: <ID>. Endereço: <STREET>.\nCódigo Postal: <ZIP>-... Nome do <SEX>: <PATIENT> NHC: <ID>. Endereço: <STREET>.\nCódigo Postal: <ZIP>. Dados de cuidados. Data de nascimento: <DATE>. Idade: <AGE> anos Data de admissão: <DATE>. \nDoutora: <DOCTOR>


#### Example 2: obfuscated

In [9]:
input_json_data = {"text": sample_text, "masking_policy": "obfuscated"}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )

pd.DataFrame(data)

,predictions
0,"Nome do Homen: Silvia Couto NHC: 1610960. Endereço: Avenida da Liberdade, 17.\nCódigo Postal: 45409-... Nome do Homen: Silvia Couto NHC: 8119147. Endereço: Avenida da Liberdade, 17.\nCódigo Postal: 82956-213. Dados de cuidados. Data de nascimento: 10/10/1963. Idade: 36 anos Data de admissão: 17/06/2016. \nDoutora: Isabel Magalhães"


#### Example 3: masked (default-policy)

In [10]:
input_json_data = {"text": docs}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )

pd.DataFrame(data)

,predictions
0,"Dados do <SEX>. Nome: <PATIENT>. Apelido: <PATIENT>. NISS: <ID>. Endereço: <STREET>.\nCÓDIGO POSTAL: <ZIP>. Dados de cuidados. Data de nascimento: <DATE>. País: <COUNTRY>.\nIdade: <AGE> anos Sexo: <SEX>.\nData de admissão: <DATE>. Doutor: <DOCTOR> Cuéllar NºCol: <ID> <ID> <ID>. Relatório clínico do <SEX>: <SEX> de <AGE> anos, mineiro reformado, sem alergias medicamentosas conhecidas, que apresenta como história pessoal: acidente de trabalho antigo com fracturas vertebrais e das costelas; operado por doença de Dupuytren na mão direita e iliofemoral esquerda; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; alcoolismo activo, fumador de 20 cigarros / dia. Foi encaminhado dos cuidados primários porque apresentou uma vez hematúria macroscópica pós-morte e depois microhaematúria persistente, com micturição normal.\nO exame físico mostrou um bom estado geral, com abdómen e genitália normais; o exame rectal foi compatível com adenoma de próstata de grau I/IV. A urinálise mostrou 4 glóbulos vermelhos/campo e 0-5 leucócitos/campo; o resto do sedimento estava normal.\nHemograma normal; a bioquímica mostrou glicemia de 169 mg/dl e triglicér<SEX> de 456 mg/dl; função hepática e renal normal. PSA de 1,16 ng/ml. A citologia da urina era repetidamente desconfiada por malignidade. A radiografia simples abdominal mostra alterações degenerativas na coluna lombar e calcificações vasculares tanto no hipocôndrio como na pélvis. A ecografia urológica revelou cistos corticais simples no rim direito, uma bexiga inalterada com boa capacidade e uma próstata com 30g de peso. O IVUS mostrou normofuncionalismo renal bilateral, calcificações na silhueta renal direita e ureteres artrosados com imagens de adição no terço superior de ambos os ureteres, relacionadas com pseudodiverticulose ureteral. O cistograma mostra uma bexiga com boa capacidade, mas com paredes trabeculadas em relação à bexiga de stress. A tomografia computorizada abdominal é normal.\nA cistoscopia revelou a existência de pequenos tumores na bexiga, e a ressecção transuretral foi realizada com o resultado anatomopatológico do carcinoma urotelial superficial da bexiga. Referido por: <DOCTOR> - <STREET>, 22 <COUNTRY> E-mail: <EMAIL>."
1,"RELAÇÃO HOSPITALAR\nNOME: <DOCTOR> NHC: <ID> ENDEREÇO: <STREET> CÓDIGO POSTA. .. RELAÇÃO HOSPITALAR\nNOME: <PATIENT> NHC: <ID> ENDEREÇO: <STREET> CÓDIGO POSTAL: <ZIP>\nDATA DE NASCIMENTO: <DATE> IDADE: <AGE> anos SEXO: <SEX> E-MAIL: <EMAIL>\nDATA DE ADMISSÃO: <DATE> DOUTORA: <DOCTOR> RELATO CLÍNICO: <AGE> anos, aposentado, sem alergia a medicamentos conhecida, com a seguinte história: ex-acidente de trabalho com fratura de vértebras e costelas; operado de doença de Dupuytren na mão direita e ponte ílio-femoral esquerda; diabetes tipo II, hipercolesterolemia e hiperuricemia; alcoolismo ativo, fuma 20 cigarros/dia. Ele foi-nos encaminhado por apresentar hematúria macroscópica pós-evacuação em uma ocasião e microhematúria persistente posteriormente, com evacuação normal.\nO exame físico mostrou bom estado geral, com abdome e genitais normais; o toque retal foi compatível com adenoma de próstata grau I/IV. A urinálise mostrou 4 hemácias/campo e 0-5 leucócitos/campo; o resto do sedimento era normal.\nO hemograma é normal; a bioquímica mostrou uma glicemia de 169 mg/dl e triglicerídeos 456 mg/dl; função hepática e renal são normais. PSA de 1,16 ng/ml. DIRIGIDA A: Dr. <DOCTOR> - Centro <HOSPITAL> - <STREET>, 56 E-MAIL: evandrade@poste. pt"


#### Example 4: obfuscated

In [11]:
input_json_data = {"text": docs, "masking_policy": "obfuscated"}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )

pd.DataFrame(data)

,predictions
0,"Dados do Homen. Nome: Josefina Batista. Apelido: Tiago Santos. NISS: 09 10960454. Endereço: Avenida da Liberdade, 17.\nCÓDIGO POSTAL: 81191. Dados de cuidados. Data de nascimento: 03/03/1946. País: Brasil.\nIdade: 36 anos Sexo: H..\nData de admissão: 12/12/2016. Doutor: Vasco Soares Cuéllar NºCol: 47 82 95621. Relatório clínico do Homen: Homen de 36 anos, mineiro reformado, sem alergias medicamentosas conhecidas, que apresenta como história pessoal: acidente de trabalho antigo com fracturas vertebrais e das costelas; operado por doença de Dupuytren na mão direita e iliofemoral esquerda; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; alcoolismo activo, fumador de 20 cigarros / dia. Foi encaminhado dos cuidados primários porque apresentou uma vez hematúria macroscópica pós-morte e depois microhaematúria persistente, com micturição normal.\nO exame físico mostrou um bom estado geral, com abdómen e genitália normais; o exame rectal foi compatível com adenoma de próstata de grau I/IV. A urinálise mostrou 4 glóbulos vermelhos/campo e 0-5 leucócitos/campo; o resto do sedimento estava normal.\nHemograma normal; a bioquímica mostrou glicemia de 169 mg/dl e triglicérH. de 456 mg/dl; função hepática e renal normal. PSA de 1,16 ng/ml. A citologia da urina era repetidamente desconfiada por malignidade. A radiografia simples abdominal mostra alterações degenerativas na coluna lombar e calcificações vasculares tanto no hipocôndrio como na pélvis. A ecografia urológica revelou cistos corticais simples no rim direito, uma bexiga inalterada com boa capacidade e uma próstata com 30g de peso. O IVUS mostrou normofuncionalismo renal bilateral, calcificações na silhueta renal direita e ureteres artrosados com imagens de adição no terço superior de ambos os ureteres, relacionadas com pseudodiverticulose ureteral. O cistograma mostra uma bexiga com boa capacidade, mas com paredes trabeculadas em relação à bexiga de stress. A tomografia computorizada abdominal é normal.\nA cistoscopia revelou a existência de pequenos tumores na bexiga, e a ressecção transuretral foi realizada com o resultado anatomopatológico do carcinoma urotelial superficial da bexiga. Referido por: Carlos Melo - Rua de Santa María, 100, 22 Brasil E-mail: eric.shannon@geegle.com."
1,"RELAÇÃO HOSPITALAR\nNOME: Vasco Soares NHC: HYQMVH84O96E952W ENDEREÇO: Rua Augusta, 19 CÓDIGO POSTA. .. RELAÇÃO HOSPITALAR\nNOME: Vasco Soares NHC: UXLKGM01U27O536U ENDEREÇO: Rua Augusta, 19 CÓDIGO POSTAL: 44034\nDATA DE NASCIMENTO: 03/03/1946 IDADE: 36 anos SEXO: Mulher E-MAIL: richard@yahoo.pt\nDATA DE ADMISSÃO: 12/12/2016 DOUTORA: Eva Coutinho RELATO CLÍNICO: 36 anos, aposentado, sem alergia a medicamentos conhecida, com a seguinte história: ex-acidente de trabalho com fratura de vértebras e costelas; operado de doença de Dupuytren na mão direita e ponte ílio-femoral esquerda; diabetes tipo II, hipercolesterolemia e hiperuricemia; alcoolismo ativo, fuma 20 cigarros/dia. Ele foi-nos encaminhado por apresentar hematúria macroscópica pós-evacuação em uma ocasião e microhematúria persistente posteriormente, com evacuação normal.\nO exame físico mostrou bom estado geral, com abdome e genitais normais; o toque retal foi compatível com adenoma de próstata grau I/IV. A urinálise mostrou 4 hemácias/campo e 0-5 leucócitos/campo; o resto do sedimento era normal.\nO hemograma é normal; a bioquímica mostrou uma glicemia de 169 mg/dl e triglicerídeos 456 mg/dl; função hepática e renal são normais. PSA de 1,16 ng/ml. DIRIGIDA A: Dr. Eva Coutinho - Centro Centro Hospitalar Universitario do Algarve - Rua de Santa María, 100, 56 E-MAIL: evandrade@poste. pt"


### JSON Lines

1. **Input format**: Provide input in JSON Lines format, where each line is a JSON object representing a text document along with any optional parameters.

```json
{"text": "Text document 1"}
{"text": "Text document 2"}
```

In [12]:
import json

def create_jsonl(records, masking_policy):
    json_records = []

    for text in records:
        record = {
            "text": text,
            "masking_policy": masking_policy
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

#### Example 1: masked (default-policy)

In [13]:
input_jsonl_data = create_jsonl(docs, masking_policy="masked")

data =  process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )

print(data)

{"predictions": "Dados do <SEX>. Nome: <PATIENT>. Apelido: <PATIENT>. NISS: <ID>. Endereço: <STREET>.\nCÓDIGO POSTAL: <ZIP>. Dados de cuidados. Data de nascimento: <DATE>. País: <COUNTRY>.\nIdade: <AGE> anos Sexo: <SEX>.\nData de admissão: <DATE>. Doutor: <DOCTOR> Cuéllar NºCol: <ID> <ID> <ID>. Relatório clínico do <SEX>: <SEX> de <AGE> anos, mineiro reformado, sem alergias medicamentosas conhecidas, que apresenta como história pessoal: acidente de trabalho antigo com fracturas vertebrais e das costelas; operado por doença de Dupuytren na mão direita e iliofemoral esquerda; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; alcoolismo activo, fumador de 20 cigarros / dia. Foi encaminhado dos cuidados primários porque apresentou uma vez hematúria macroscópica pós-morte e depois microhaematúria persistente, com micturição normal.\nO exame físico mostrou um bom estado geral, com abdómen e genitália normais; o exame rectal foi compatível com adenoma de próstata de grau I/IV. A

#### Example 2: obfuscated

In [14]:
input_jsonl_data = create_jsonl(docs, masking_policy="obfuscated")

data =  process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )

print(data)

{"predictions": "Dados do Homen. Nome: Josefina Batista. Apelido: Tiago Santos. NISS: 09 10960454. Endereço: Avenida da Liberdade, 17.\nCÓDIGO POSTAL: 81191. Dados de cuidados. Data de nascimento: 03/03/1946. País: Brasil.\nIdade: 36 anos Sexo: H..\nData de admissão: 12/12/2016. Doutor: Vasco Soares Cuéllar NºCol: 47 82 95621. Relatório clínico do Homen: Homen de 36 anos, mineiro reformado, sem alergias medicamentosas conhecidas, que apresenta como história pessoal: acidente de trabalho antigo com fracturas vertebrais e das costelas; operado por doença de Dupuytren na mão direita e iliofemoral esquerda; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; alcoolismo activo, fumador de 20 cigarros / dia. Foi encaminhado dos cuidados primários porque apresentou uma vez hematúria macroscópica pós-morte e depois microhaematúria persistente, com micturição normal.\nO exame físico mostrou um bom estado geral, com abdómen e genitália normais; o exame rectal foi compatível com adeno

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [15]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [16]:
import json
import os


input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)


validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"


def write_and_upload_to_s3(input_data, file_name):

    file_format = os.path.splitext(file_name)[1].lower()
    s3_key = f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}"

    if file_format == ".json":
        input_data_json = json.dumps(input_data, indent=4, ensure_ascii=False)

        with open(file_name, "w", encoding='utf-8') as f:
            f.write(input_data_json)
    elif file_format == ".jsonl":
        with open(file_name, 'w', encoding='utf-8') as file:
            for line in input_data.splitlines():
                json_object = json.loads(line)
                file.write(json.dumps(json_object, ensure_ascii=False) + '\n')

    with open(file_name, 'rb') as data:
        s3_client.put_object(Bucket=s3_bucket, Key=s3_key, Body=data)

    return input_data


input_json_data = {
    "input1.json": {"text": docs, "masking_policy": "masked"},
    "input2.json": {"text": docs, "masking_policy": "obfuscated"},
    "input3.json": {"text": docs, "masking_policy": "masked_fixed_length_chars"},
    "input4.json": {"text": docs, "masking_policy": "masked_with_chars"}
}

input_jsonl_data = {
    "input1.jsonl": create_jsonl(docs, masking_policy="masked"),
    "input2.jsonl": create_jsonl(docs, masking_policy="obfuscated"),
    "input3.jsonl": create_jsonl(docs, masking_policy="masked_fixed_length_chars"),
    "input4.jsonl": create_jsonl(docs, masking_policy="masked_with_chars")
}


for file_name, data in input_json_data.items():
    write_and_upload_to_s3(data, f"{json_input_dir}/{file_name}")

for file_name, data in input_jsonl_data.items():
    write_and_upload_to_s3(data, f"{jsonl_input_dir}/{file_name}")



### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [18]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):
    
    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data)
    display(df)

    with open(output_file_path, 'w', encoding='utf-8') as f_out:
        json.dump(data, f_out, indent=4, ensure_ascii=False)

In [19]:
masking_policies = {
    "masked": "input1.json",
    "obfuscated": "input2.json",
    "masked_fixed_length_chars": "input3.json",
    "masked_with_chars": "input4.json",
}

for policy_name, validation_file_name in masking_policies.items():
    print(f"Masking policy: {policy_name}")
    process_s3_json_output_and_save(validation_file_name)
    print("\n")

Masking policy: masked


,predictions
0,"Dados do <SEX>. Nome: <PATIENT>. Apelido: <PATIENT>. NISS: <ID>. Endereço: <STREET>.\nCÓDIGO POSTAL: <ZIP>. Dados de cuidados. Data de nascimento: <DATE>. País: <COUNTRY>.\nIdade: <AGE> anos Sexo: <SEX>.\nData de admissão: <DATE>. Doutor: <DOCTOR> Cuéllar NºCol: <ID> <ID> <ID>. Relatório clínico do <SEX>: <SEX> de <AGE> anos, mineiro reformado, sem alergias medicamentosas conhecidas, que apresenta como história pessoal: acidente de trabalho antigo com fracturas vertebrais e das costelas; operado por doença de Dupuytren na mão direita e iliofemoral esquerda; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; alcoolismo activo, fumador de 20 cigarros / dia. Foi encaminhado dos cuidados primários porque apresentou uma vez hematúria macroscópica pós-morte e depois microhaematúria persistente, com micturição normal.\nO exame físico mostrou um bom estado geral, com abdómen e genitália normais; o exame rectal foi compatível com adenoma de próstata de grau I/IV. A urinálise mostrou 4 glóbulos vermelhos/campo e 0-5 leucócitos/campo; o resto do sedimento estava normal.\nHemograma normal; a bioquímica mostrou glicemia de 169 mg/dl e triglicér<SEX> de 456 mg/dl; função hepática e renal normal. PSA de 1,16 ng/ml. A citologia da urina era repetidamente desconfiada por malignidade. A radiografia simples abdominal mostra alterações degenerativas na coluna lombar e calcificações vasculares tanto no hipocôndrio como na pélvis. A ecografia urológica revelou cistos corticais simples no rim direito, uma bexiga inalterada com boa capacidade e uma próstata com 30g de peso. O IVUS mostrou normofuncionalismo renal bilateral, calcificações na silhueta renal direita e ureteres artrosados com imagens de adição no terço superior de ambos os ureteres, relacionadas com pseudodiverticulose ureteral. O cistograma mostra uma bexiga com boa capacidade, mas com paredes trabeculadas em relação à bexiga de stress. A tomografia computorizada abdominal é normal.\nA cistoscopia revelou a existência de pequenos tumores na bexiga, e a ressecção transuretral foi realizada com o resultado anatomopatológico do carcinoma urotelial superficial da bexiga. Referido por: <DOCTOR> - <STREET>, 22 <COUNTRY> E-mail: <EMAIL>."
1,"RELAÇÃO HOSPITALAR\nNOME: <DOCTOR> NHC: <ID> ENDEREÇO: <STREET> CÓDIGO POSTA. .. RELAÇÃO HOSPITALAR\nNOME: <PATIENT> NHC: <ID> ENDEREÇO: <STREET> CÓDIGO POSTAL: <ZIP>\nDATA DE NASCIMENTO: <DATE> IDADE: <AGE> anos SEXO: <SEX> E-MAIL: <EMAIL>\nDATA DE ADMISSÃO: <DATE> DOUTORA: <DOCTOR> RELATO CLÍNICO: <AGE> anos, aposentado, sem alergia a medicamentos conhecida, com a seguinte história: ex-acidente de trabalho com fratura de vértebras e costelas; operado de doença de Dupuytren na mão direita e ponte ílio-femoral esquerda; diabetes tipo II, hipercolesterolemia e hiperuricemia; alcoolismo ativo, fuma 20 cigarros/dia. Ele foi-nos encaminhado por apresentar hematúria macroscópica pós-evacuação em uma ocasião e microhematúria persistente posteriormente, com evacuação normal.\nO exame físico mostrou bom estado geral, com abdome e genitais normais; o toque retal foi compatível com adenoma de próstata grau I/IV. A urinálise mostrou 4 hemácias/campo e 0-5 leucócitos/campo; o resto do sedimento era normal.\nO hemograma é normal; a bioquímica mostrou uma glicemia de 169 mg/dl e triglicerídeos 456 mg/dl; função hepática e renal são normais. PSA de 1,16 ng/ml. DIRIGIDA A: Dr. <DOCTOR> - Centro <HOSPITAL> - <STREET>, 56 E-MAIL: evandrade@poste. pt"




Masking policy: obfuscated


,predictions
0,"Dados do Homen. Nome: Josefina Batista. Apelido: Tiago Santos. NISS: 09 10960454. Endereço: Avenida da Liberdade, 17.\nCÓDIGO POSTAL: 81191. Dados de cuidados. Data de nascimento: 03/03/1946. País: Brasil.\nIdade: 36 anos Sexo: H..\nData de admissão: 12/12/2016. Doutor: Vasco Soares Cuéllar NºCol: 47 82 95621. Relatório clínico do Homen: Homen de 36 anos, mineiro reformado, sem alergias medicamentosas conhecidas, que apresenta como história pessoal: acidente de trabalho antigo com fracturas vertebrais e das costelas; operado por doença de Dupuytren na mão direita e iliofemoral esquerda; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; alcoolismo activo, fumador de 20 cigarros / dia. Foi encaminhado dos cuidados primários porque apresentou uma vez hematúria macroscópica pós-morte e depois microhaematúria persistente, com micturição normal.\nO exame físico mostrou um bom estado geral, com abdómen e genitália normais; o exame rectal foi compatível com adenoma de próstata de grau I/IV. A urinálise mostrou 4 glóbulos vermelhos/campo e 0-5 leucócitos/campo; o resto do sedimento estava normal.\nHemograma normal; a bioquímica mostrou glicemia de 169 mg/dl e triglicérH. de 456 mg/dl; função hepática e renal normal. PSA de 1,16 ng/ml. A citologia da urina era repetidamente desconfiada por malignidade. A radiografia simples abdominal mostra alterações degenerativas na coluna lombar e calcificações vasculares tanto no hipocôndrio como na pélvis. A ecografia urológica revelou cistos corticais simples no rim direito, uma bexiga inalterada com boa capacidade e uma próstata com 30g de peso. O IVUS mostrou normofuncionalismo renal bilateral, calcificações na silhueta renal direita e ureteres artrosados com imagens de adição no terço superior de ambos os ureteres, relacionadas com pseudodiverticulose ureteral. O cistograma mostra uma bexiga com boa capacidade, mas com paredes trabeculadas em relação à bexiga de stress. A tomografia computorizada abdominal é normal.\nA cistoscopia revelou a existência de pequenos tumores na bexiga, e a ressecção transuretral foi realizada com o resultado anatomopatológico do carcinoma urotelial superficial da bexiga. Referido por: Carlos Melo - Rua de Santa María, 100, 22 Brasil E-mail: eric.shannon@geegle.com."
1,"RELAÇÃO HOSPITALAR\nNOME: Vasco Soares NHC: ZOXWRU04V40J811B ENDEREÇO: Rua Augusta, 19 CÓDIGO POSTA. .. RELAÇÃO HOSPITALAR\nNOME: Vasco Soares NHC: JYNWGN56O13Y865H ENDEREÇO: Rua Augusta, 19 CÓDIGO POSTAL: 84696\nDATA DE NASCIMENTO: 03/03/1946 IDADE: 36 anos SEXO: Mulher E-MAIL: richard@yahoo.pt\nDATA DE ADMISSÃO: 12/12/2016 DOUTORA: Eva Coutinho RELATO CLÍNICO: 36 anos, aposentado, sem alergia a medicamentos conhecida, com a seguinte história: ex-acidente de trabalho com fratura de vértebras e costelas; operado de doença de Dupuytren na mão direita e ponte ílio-femoral esquerda; diabetes tipo II, hipercolesterolemia e hiperuricemia; alcoolismo ativo, fuma 20 cigarros/dia. Ele foi-nos encaminhado por apresentar hematúria macroscópica pós-evacuação em uma ocasião e microhematúria persistente posteriormente, com evacuação normal.\nO exame físico mostrou bom estado geral, com abdome e genitais normais; o toque retal foi compatível com adenoma de próstata grau I/IV. A urinálise mostrou 4 hemácias/campo e 0-5 leucócitos/campo; o resto do sedimento era normal.\nO hemograma é normal; a bioquímica mostrou uma glicemia de 169 mg/dl e triglicerídeos 456 mg/dl; função hepática e renal são normais. PSA de 1,16 ng/ml. DIRIGIDA A: Dr. Eva Coutinho - Centro Centro Hospitalar Universitario do Algarve - Rua de Santa María, 100, 56 E-MAIL: evandrade@poste. pt"




Masking policy: masked_fixed_length_chars


,predictions
0,"Dados do ****. Nome: ****. Apelido: ****. NISS: ****. Endereço: ****.\nCÓDIGO POSTAL: ****. Dados de cuidados. Data de nascimento: ****. País: ****.\nIdade: **** anos Sexo: ****.\nData de admissão: ****. Doutor: **** Cuéllar NºCol: **** **** ****. Relatório clínico do ****: **** de **** anos, mineiro reformado, sem alergias medicamentosas conhecidas, que apresenta como história pessoal: acidente de trabalho antigo com fracturas vertebrais e das costelas; operado por doença de Dupuytren na mão direita e iliofemoral esquerda; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; alcoolismo activo, fumador de 20 cigarros / dia. Foi encaminhado dos cuidados primários porque apresentou uma vez hematúria macroscópica pós-morte e depois microhaematúria persistente, com micturição normal.\nO exame físico mostrou um bom estado geral, com abdómen e genitália normais; o exame rectal foi compatível com adenoma de próstata de grau I/IV. A urinálise mostrou 4 glóbulos vermelhos/campo e 0-5 leucócitos/campo; o resto do sedimento estava normal.\nHemograma normal; a bioquímica mostrou glicemia de 169 mg/dl e triglicér**** de 456 mg/dl; função hepática e renal normal. PSA de 1,16 ng/ml. A citologia da urina era repetidamente desconfiada por malignidade. A radiografia simples abdominal mostra alterações degenerativas na coluna lombar e calcificações vasculares tanto no hipocôndrio como na pélvis. A ecografia urológica revelou cistos corticais simples no rim direito, uma bexiga inalterada com boa capacidade e uma próstata com 30g de peso. O IVUS mostrou normofuncionalismo renal bilateral, calcificações na silhueta renal direita e ureteres artrosados com imagens de adição no terço superior de ambos os ureteres, relacionadas com pseudodiverticulose ureteral. O cistograma mostra uma bexiga com boa capacidade, mas com paredes trabeculadas em relação à bexiga de stress. A tomografia computorizada abdominal é normal.\nA cistoscopia revelou a existência de pequenos tumores na bexiga, e a ressecção transuretral foi realizada com o resultado anatomopatológico do carcinoma urotelial superficial da bexiga. Referido por: **** - ****, 22 **** E-mail: ****."
1,"RELAÇÃO HOSPITALAR\nNOME: **** NHC: **** ENDEREÇO: **** CÓDIGO POSTA. .. RELAÇÃO HOSPITALAR\nNOME: **** NHC: **** ENDEREÇO: **** CÓDIGO POSTAL: ****\nDATA DE NASCIMENTO: **** IDADE: **** anos SEXO: **** E-MAIL: ****\nDATA DE ADMISSÃO: **** DOUTORA: **** RELATO CLÍNICO: **** anos, aposentado, sem alergia a medicamentos conhecida, com a seguinte história: ex-acidente de trabalho com fratura de vértebras e costelas; operado de doença de Dupuytren na mão direita e ponte ílio-femoral esquerda; diabetes tipo II, hipercolesterolemia e hiperuricemia; alcoolismo ativo, fuma 20 cigarros/dia. Ele foi-nos encaminhado por apresentar hematúria macroscópica pós-evacuação em uma ocasião e microhematúria persistente posteriormente, com evacuação normal.\nO exame físico mostrou bom estado geral, com abdome e genitais normais; o toque retal foi compatível com adenoma de próstata grau I/IV. A urinálise mostrou 4 hemácias/campo e 0-5 leucócitos/campo; o resto do sedimento era normal.\nO hemograma é normal; a bioquímica mostrou uma glicemia de 169 mg/dl e triglicerídeos 456 mg/dl; função hepática e renal são normais. PSA de 1,16 ng/ml. DIRIGIDA A: Dr. **** - Centro **** - ****, 56 E-MAIL: evandrade@poste. pt"




Masking policy: masked_with_chars


,predictions
0,"Dados do [******]. Nome: [***]. Apelido: [*******]. NISS: **********]. Endereço: [*********************].\nCÓDIGO POSTAL: [***]. Dados de cuidados. Data de nascimento: [********]. País: [******].\nIdade: ** anos Sexo: *.\nData de admissão: [********]. Doutor: [*************] Cuéllar NºCol: ** ** [***]. Relatório clínico do [******]: [******] de ** anos, mineiro reformado, sem alergias medicamentosas conhecidas, que apresenta como história pessoal: acidente de trabalho antigo com fracturas vertebrais e das costelas; operado por doença de Dupuytren na mão direita e iliofemoral esquerda; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; alcoolismo activo, fumador de 20 cigarros / dia. Foi encaminhado dos cuidados primários porque apresentou uma vez hematúria macroscópica pós-morte e depois microhaematúria persistente, com micturição normal.\nO exame físico mostrou um bom estado geral, com abdómen e genitália normais; o exame rectal foi compatível com adenoma de próstata de grau I/IV. A urinálise mostrou 4 glóbulos vermelhos/campo e 0-5 leucócitos/campo; o resto do sedimento estava normal.\nHemograma normal; a bioquímica mostrou glicemia de 169 mg/dl e triglicér[**] de 456 mg/dl; função hepática e renal normal. PSA de 1,16 ng/ml. A citologia da urina era repetidamente desconfiada por malignidade. A radiografia simples abdominal mostra alterações degenerativas na coluna lombar e calcificações vasculares tanto no hipocôndrio como na pélvis. A ecografia urológica revelou cistos corticais simples no rim direito, uma bexiga inalterada com boa capacidade e uma próstata com 30g de peso. O IVUS mostrou normofuncionalismo renal bilateral, calcificações na silhueta renal direita e ureteres artrosados com imagens de adição no terço superior de ambos os ureteres, relacionadas com pseudodiverticulose ureteral. O cistograma mostra uma bexiga com boa capacidade, mas com paredes trabeculadas em relação à bexiga de stress. A tomografia computorizada abdominal é normal.\nA cistoscopia revelou a existência de pequenos tumores na bexiga, e a ressecção transuretral foi realizada com o resultado anatomopatológico do carcinoma urotelial superficial da bexiga. Referido por: [***********] - [*****************], 22 [******] E-mail: [****************]."
1,"RELAÇÃO HOSPITALAR\nNOME: [*************] NHC: [**************] ENDEREÇO: [************] CÓDIGO POSTA. .. RELAÇÃO HOSPITALAR\nNOME: [*************] NHC: [**************] ENDEREÇO: [************] CÓDIGO POSTAL: [***]\nDATA DE NASCIMENTO: [********] IDADE: ** anos SEXO: [****] E-MAIL: [***********]\nDATA DE ADMISSÃO: [********] DOUTORA: [*********] RELATO CLÍNICO: ** anos, aposentado, sem alergia a medicamentos conhecida, com a seguinte história: ex-acidente de trabalho com fratura de vértebras e costelas; operado de doença de Dupuytren na mão direita e ponte ílio-femoral esquerda; diabetes tipo II, hipercolesterolemia e hiperuricemia; alcoolismo ativo, fuma 20 cigarros/dia. Ele foi-nos encaminhado por apresentar hematúria macroscópica pós-evacuação em uma ocasião e microhematúria persistente posteriormente, com evacuação normal.\nO exame físico mostrou bom estado geral, com abdome e genitais normais; o toque retal foi compatível com adenoma de próstata grau I/IV. A urinálise mostrou 4 hemácias/campo e 0-5 leucócitos/campo; o resto do sedimento era normal.\nO hemograma é normal; a bioquímica mostrou uma glicemia de 169 mg/dl e triglicerídeos 456 mg/dl; função hepática e renal são normais. PSA de 1,16 ng/ml. DIRIGIDA A: Dr. [*********] - Centro [*********************] - [*****************], 56 E-MAIL: evandrade@poste. pt"


### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [21]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [22]:
masking_policies = {
    "masked": "input1.jsonl",
    "obfuscated": "input2.jsonl",
    "masked_fixed_length_chars": "input3.jsonl",
    "masked_with_chars": "input4.jsonl",
}

for policy_name, validation_file_name in masking_policies.items():
    print(f"Masking policy: {policy_name}")
    process_s3_jsonlines_output_and_save(validation_file_name)
    print("\n")

Masking policy: masked
{"predictions": "Dados do <SEX>. Nome: <PATIENT>. Apelido: <PATIENT>. NISS: <ID>. Endereço: <STREET>.\nCÓDIGO POSTAL: <ZIP>. Dados de cuidados. Data de nascimento: <DATE>. País: <COUNTRY>.\nIdade: <AGE> anos Sexo: <SEX>.\nData de admissão: <DATE>. Doutor: <DOCTOR> Cuéllar NºCol: <ID> <ID> <ID>. Relatório clínico do <SEX>: <SEX> de <AGE> anos, mineiro reformado, sem alergias medicamentosas conhecidas, que apresenta como história pessoal: acidente de trabalho antigo com fracturas vertebrais e das costelas; operado por doença de Dupuytren na mão direita e iliofemoral esquerda; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; alcoolismo activo, fumador de 20 cigarros / dia. Foi encaminhado dos cuidados primários porque apresentou uma vez hematúria macroscópica pós-morte e depois microhaematúria persistente, com micturição normal.\nO exame físico mostrou um bom estado geral, com abdómen e genitália normais; o exame rectal foi compatível com adenoma de p

In [23]:
model.delete_model()

INFO:sagemaker:Deleting model with name: pt-deid-clinical-2024-05-21-07-08-12-366


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

